#### Write a web scraping script which takes an input of any film actor and gives the output of filmography of that actor in descending order.

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def get_actor_filmography(actor_name):
    # Search IMDb for the actor
    search_url = "https://www.imdb.com/find"
    params = {'q': actor_name, 's': 'nm'}  # Search for names
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    search_response = requests.get(search_url, params=params, headers=headers)
    search_soup = BeautifulSoup(search_response.content, 'html.parser')
    
    # Find the first matching actor result (change as needed)
    actor_link = search_soup.find('a', href=re.compile('/name/nm'))
    if not actor_link:
        print("Actor not found.")
        return
    
    actor_url = "https://www.imdb.com" + actor_link['href']
    
    # Request the actor's main page
    actor_response = requests.get(actor_url, headers=headers)
    actor_soup = BeautifulSoup(actor_response.content, 'html.parser')
    
    # Locate the filmography section
    filmography = actor_soup.find_all('div', class_='filmo-row')
    films = []
    
    for film in filmography:
        title_year = film.find('span', class_='year_column')
        title = film.find('a')
        
        if title and title_year:
            # Extract title and year, stripping out any extra whitespace
            title_text = title.get_text(strip=True)
            year_text = title_year.get_text(strip=True)
            
            # Some entries don't have a year, skip these
            if year_text:
                try:
                    year = int(year_text[:4])  # Convert year to int for sorting
                    films.append((title_text, year))
                except ValueError:
                    continue  # Skip entries without a valid year

    # Sort films by year in descending order
    films.sort(key=lambda x: x[1], reverse=True)
    
    # Print the results
    print(f"Filmography of {actor_name} (in descending order):")
    for title, year in films:
        print(f"{year}: {title}")

# Input actor name
actor_name = input("Enter the actor's name: ")
get_actor_filmography(actor_name)
